In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from nflows.transforms.dequantization import UniformDequantization
from nflows.transforms.dequantization import VariationalDequantization

In [4]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [5]:
# Make model folder
if not os.path.exists("models"):
    os.makedirs("models")

In [6]:
# Import data and weights
x_data_raw = torch.tensor(pd.read_csv("ee_data.csv", header=None, delimiter=",").to_numpy(), dtype=torch.float32, device=device)
x_weights_raw = torch.tensor(pd.read_csv("ee_weights.csv", header=None, delimiter=",").to_numpy(), dtype=torch.float32, device=device).squeeze()

# Permute the data
permutation = torch.randperm(x_data_raw.shape[0])
x_data_raw = x_data_raw[permutation]
x_weights_raw = x_weights_raw[permutation]

# Normalize weights by mean
x_weights_raw /= x_weights_raw.mean()

# Chop up the data into training and validation
data_size = x_data_raw.shape[0]
training_size = int(data_size*0.8)

x_data_train    = x_data_raw[:training_size]
x_weights_train = x_weights_raw[:training_size]
x_data_test     = x_data_raw[training_size:]
x_weights_test  = x_weights_raw[training_size:]

# Data dimension
data_dim = x_data_raw.shape[1]

In [7]:
# Determine the maximum labels of discrete dims
max_features = torch.max(x_data_raw, dim=0)[0]
is_discrete = max_features > 1
max_labels = torch.where(max_features > 1, max_features, torch.tensor(-1., device=device))

In [8]:
# Tensorboard writer
writer = SummaryWriter()

In [9]:
num_layers = 6
base_dist_uniform = BoxUniform(torch.zeros(data_dim), torch.ones(data_dim))
base_dist_variational = BoxUniform(torch.zeros(data_dim), torch.ones(data_dim))

transforms_uniform = []
transforms_variational = []

transforms_uniform.append(UniformDequantization(max_labels=max_labels))
transforms_variational.append(VariationalDequantization(max_labels=max_labels, rqs_hidden_features=15, rqs_flow_layers=2))

for _ in range(num_layers):
    transforms_uniform.append(RandomPermutation(features=data_dim))
    transforms_uniform.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=data_dim, 
        hidden_features=26,
        num_bins=10,
        num_blocks=4,
    ))

    transforms_variational.append(RandomPermutation(features=data_dim))
    transforms_variational.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=data_dim, 
        hidden_features=25,
        num_bins=10,
        num_blocks=4,
    ))

transform_uniform = CompositeTransform(transforms_uniform)
transform_variational = CompositeTransform(transforms_variational)

flow_uniform = Flow(transform_uniform, base_dist_uniform).to(device)
flow_variational = Flow(transform_variational, base_dist_variational).to(device)

optimizer_uniform = optim.Adam(flow_uniform.parameters())
optimizer_variational = optim.Adam(flow_variational.parameters())

In [10]:
# Number of parameters in uniform and variational models
uniform_parameters = filter(lambda p: p.requires_grad, flow_uniform.parameters())
variational_parameters = filter(lambda p: p.requires_grad, flow_variational.parameters())
num_uniform_parameters = sum([np.prod(p.size()) for p in uniform_parameters])
num_variational_parameters = sum([np.prod(p.size()) for p in variational_parameters])

print(num_uniform_parameters, num_variational_parameters)


54564 54778


In [11]:
n_epochs = 200
batch_size = 10000
n_batches = m.ceil(x_data_train.shape[0]/batch_size)

for epoch in range(n_epochs):
    permutation = torch.randperm(x_data_train.shape[0], device=device)    

    # Loop over batches
    cum_loss_uniform = 0
    cum_loss_variational = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, x_data_train.shape[0]-1 )
        indices = permutation[batch_begin:batch_end]
        x_data_train_batch = x_data_train[indices]
        x_weights_train_batch = x_weights_train[indices]
        
        # Take a step
        optimizer_uniform.zero_grad()
        optimizer_variational.zero_grad()

        loss_uniform = -(flow_uniform.log_prob(inputs=x_data_train_batch)*x_weights_train_batch).mean()
        loss_variational = -(flow_variational.log_prob(inputs=x_data_train_batch)*x_weights_train_batch).mean()

        loss_uniform.backward()
        loss_variational.backward()

        optimizer_uniform.step()
        optimizer_variational.step()

        # Compute cumulative loss
        cum_loss_uniform = (cum_loss_uniform*batch + loss_uniform.item())/(batch+1)
        cum_loss_variational = (cum_loss_variational*batch + loss_variational.item())/(batch+1)
        
        if batch%10 == 0:
            print("epoch = ", epoch, "batch = ",batch+1, "/", n_batches, "loss_uniform = ", cum_loss_uniform, " loss_variational = ", cum_loss_variational)
    
    # Validation log prob
    #loss_uniform_validation     = -(flow_uniform.log_prob(inputs=x_data_test)*x_weights_test).mean()
    #loss_variational_validation = -(flow_variational.log_prob(inputs=x_data_test)*x_weights_test).mean()

    writer.add_scalar("Loss_train/uniform_loss_train", cum_loss_uniform, epoch)
    writer.add_scalar("Loss_train/variational_loss_train", cum_loss_variational, epoch)

    #writer.add_scalar("Loss_test/uniform_loss_test", loss_uniform_validation, epoch)
    #writer.add_scalar("Loss_test/variational_loss_test", loss_variational_validation, epoch)

    torch.save(flow_uniform, "models/uniform_model.pt")
    torch.save(flow_variational, "models/variational_model.pt")

epoch =  0 batch =  1 / 101 loss_uniform =  1.039359211921692  loss_variational =  3.0154170989990234
epoch =  0 batch =  11 / 101 loss_uniform =  -0.4221790592101487  loss_variational =  0.727385669269345
epoch =  0 batch =  21 / 101 loss_uniform =  -1.3507582628656  loss_variational =  -0.6958527525975591
epoch =  0 batch =  31 / 101 loss_uniform =  -2.1927127260113917  loss_variational =  -1.6664091206846698
epoch =  0 batch =  41 / 101 loss_uniform =  -2.894826833703896  loss_variational =  -2.347036150352257
epoch =  0 batch =  51 / 101 loss_uniform =  -3.4124934742701987  loss_variational =  -2.8667809058930365
epoch =  0 batch =  61 / 101 loss_uniform =  -3.82716992833331  loss_variational =  -3.3034394829243916
epoch =  0 batch =  71 / 101 loss_uniform =  -4.160175432569124  loss_variational =  -3.6636405315407563
epoch =  0 batch =  81 / 101 loss_uniform =  -4.44180588805933  loss_variational =  -3.9787511815443453
epoch =  0 batch =  91 / 101 loss_uniform =  -4.67704908791315

epoch =  7 batch =  21 / 101 loss_uniform =  -6.7856463477725075  loss_variational =  -7.110301721663702
epoch =  7 batch =  31 / 101 loss_uniform =  -6.784555681290165  loss_variational =  -7.075018129041118
epoch =  7 batch =  41 / 101 loss_uniform =  -6.838515688733357  loss_variational =  -7.102017274717006
epoch =  7 batch =  51 / 101 loss_uniform =  -6.860841395808201  loss_variational =  -7.1079448625153185
epoch =  7 batch =  61 / 101 loss_uniform =  -6.876746552889465  loss_variational =  -7.113179488260238
epoch =  7 batch =  71 / 101 loss_uniform =  -6.891586216402725  loss_variational =  -7.119035546208771
epoch =  7 batch =  81 / 101 loss_uniform =  -6.910512376714636  loss_variational =  -7.129239317811566
epoch =  7 batch =  91 / 101 loss_uniform =  -6.919445734757644  loss_variational =  -7.131008672190236
epoch =  7 batch =  101 / 101 loss_uniform =  -6.861651198108597  loss_variational =  -7.066060063567492
epoch =  8 batch =  1 / 101 loss_uniform =  -7.41647005081176

epoch =  14 batch =  41 / 101 loss_uniform =  -6.41842538554494  loss_variational =  -7.024178039736864
epoch =  14 batch =  51 / 101 loss_uniform =  -6.525918567881865  loss_variational =  -7.06086025986017
epoch =  14 batch =  61 / 101 loss_uniform =  -6.611629126501865  loss_variational =  -7.091600848025963
epoch =  14 batch =  71 / 101 loss_uniform =  -6.682871214101012  loss_variational =  -7.120682649209466
epoch =  14 batch =  81 / 101 loss_uniform =  -6.738204543973192  loss_variational =  -7.143464506408314
epoch =  14 batch =  91 / 101 loss_uniform =  -6.78348910677564  loss_variational =  -7.1603276912982645
epoch =  14 batch =  101 / 101 loss_uniform =  -6.805865061165083  loss_variational =  -7.171474626748869
epoch =  15 batch =  1 / 101 loss_uniform =  -6.836126327514648  loss_variational =  -7.012639045715332
epoch =  15 batch =  11 / 101 loss_uniform =  -6.655221288854426  loss_variational =  -6.519225077195601
epoch =  15 batch =  21 / 101 loss_uniform =  -6.74365490

epoch =  21 batch =  51 / 101 loss_uniform =  -7.322493440964642  loss_variational =  -7.390672384523878
epoch =  21 batch =  61 / 101 loss_uniform =  -7.340564876306252  loss_variational =  -7.406331875285164
epoch =  21 batch =  71 / 101 loss_uniform =  -7.327074151643565  loss_variational =  -7.392808578383755
epoch =  21 batch =  81 / 101 loss_uniform =  -7.321454154120551  loss_variational =  -7.387681201652244
epoch =  21 batch =  91 / 101 loss_uniform =  -7.320013020064804  loss_variational =  -7.385328539125212
epoch =  21 batch =  101 / 101 loss_uniform =  -7.241978583949627  loss_variational =  -7.307405410426678
epoch =  22 batch =  1 / 101 loss_uniform =  -7.4455437660217285  loss_variational =  -7.572098255157471
epoch =  22 batch =  11 / 101 loss_uniform =  -7.322871988469904  loss_variational =  -7.421155366030606
epoch =  22 batch =  21 / 101 loss_uniform =  -7.297795136769612  loss_variational =  -7.389173098972866
epoch =  22 batch =  31 / 101 loss_uniform =  -7.30463

epoch =  28 batch =  61 / 101 loss_uniform =  -7.353679961845523  loss_variational =  -7.403968451453037
epoch =  28 batch =  71 / 101 loss_uniform =  -7.356687666664661  loss_variational =  -7.404500793403303
epoch =  28 batch =  81 / 101 loss_uniform =  -7.361766497294108  loss_variational =  -7.405957398591219
epoch =  28 batch =  91 / 101 loss_uniform =  -7.356721238775568  loss_variational =  -7.402060016170963
epoch =  28 batch =  101 / 101 loss_uniform =  -7.517218877773474  loss_variational =  -7.561400989494701
epoch =  29 batch =  1 / 101 loss_uniform =  -6.8059234619140625  loss_variational =  -6.869312286376953
epoch =  29 batch =  11 / 101 loss_uniform =  -6.663770719008013  loss_variational =  -6.818626360459761
epoch =  29 batch =  21 / 101 loss_uniform =  -6.827008769625709  loss_variational =  -6.99235150927589
epoch =  29 batch =  31 / 101 loss_uniform =  -6.908705249909432  loss_variational =  -7.068680132589033
epoch =  29 batch =  41 / 101 loss_uniform =  -6.976074

...

epoch =  171 batch =  11 / 101 loss_uniform =  -7.457661368630149  loss_variational =  -7.439451564442027
epoch =  171 batch =  21 / 101 loss_uniform =  -7.465119793301537  loss_variational =  -7.4474135580517
epoch =  171 batch =  31 / 101 loss_uniform =  -7.447474018219979  loss_variational =  -7.4297140029168895
epoch =  171 batch =  41 / 101 loss_uniform =  -7.444770196589028  loss_variational =  -7.43164232300549
epoch =  171 batch =  51 / 101 loss_uniform =  -7.43800687789917  loss_variational =  -7.426338897031896
epoch =  171 batch =  61 / 101 loss_uniform =  -7.424772184403216  loss_variational =  -7.414505911655113
epoch =  171 batch =  71 / 101 loss_uniform =  -7.434064549459538  loss_variational =  -7.42425531736562
epoch =  171 batch =  81 / 101 loss_uniform =  -7.448674025358977  loss_variational =  -7.438159972061346
epoch =  171 batch =  91 / 101 loss_uniform =  -7.463545862134996  loss_variational =  -7.452216295095591
epoch =  171 batch =  101 / 101 loss_uniform =  -7

epoch =  178 batch =  21 / 101 loss_uniform =  -7.466577847798665  loss_variational =  -7.4430866695585705
epoch =  178 batch =  31 / 101 loss_uniform =  -7.423993356766239  loss_variational =  -7.391540896508001
epoch =  178 batch =  41 / 101 loss_uniform =  -7.4404080786356115  loss_variational =  -7.401588160817216
epoch =  178 batch =  51 / 101 loss_uniform =  -7.405391281726313  loss_variational =  -7.361982233384076
epoch =  178 batch =  61 / 101 loss_uniform =  -7.424645947628334  loss_variational =  -7.378542814098421
epoch =  178 batch =  71 / 101 loss_uniform =  -7.444310430070044  loss_variational =  -7.395793807338661
epoch =  178 batch =  81 / 101 loss_uniform =  -7.446136433401225  loss_variational =  -7.399062551098106
epoch =  178 batch =  91 / 101 loss_uniform =  -7.4501468899485825  loss_variational =  -7.4034120329133755
epoch =  178 batch =  101 / 101 loss_uniform =  -7.433414969113794  loss_variational =  -7.385224153499792
epoch =  179 batch =  1 / 101 loss_unifor

epoch =  185 batch =  31 / 101 loss_uniform =  -7.49330965165169  loss_variational =  -7.433893080680601
epoch =  185 batch =  41 / 101 loss_uniform =  -7.5007137554447825  loss_variational =  -7.443253156615467
epoch =  185 batch =  51 / 101 loss_uniform =  -7.503905735763849  loss_variational =  -7.450237152623195
epoch =  185 batch =  61 / 101 loss_uniform =  -7.469805811272293  loss_variational =  -7.420088924345423
epoch =  185 batch =  71 / 101 loss_uniform =  -7.454098479848512  loss_variational =  -7.407083430760343
epoch =  185 batch =  81 / 101 loss_uniform =  -7.45509676285732  loss_variational =  -7.40986227106165
epoch =  185 batch =  91 / 101 loss_uniform =  -7.450374388432765  loss_variational =  -7.407080582209995
epoch =  185 batch =  101 / 101 loss_uniform =  -7.552681592431399  loss_variational =  -7.51521357923451
epoch =  186 batch =  1 / 101 loss_uniform =  -7.48994255065918  loss_variational =  -7.432224750518799
epoch =  186 batch =  11 / 101 loss_uniform =  -7.

epoch =  192 batch =  41 / 101 loss_uniform =  -7.3232860332582055  loss_variational =  -7.2415303486149485
epoch =  192 batch =  51 / 101 loss_uniform =  -7.353232982111912  loss_variational =  -7.2865345618304085
epoch =  192 batch =  61 / 101 loss_uniform =  -7.384973096065834  loss_variational =  -7.327257031300029
epoch =  192 batch =  71 / 101 loss_uniform =  -7.386563777923584  loss_variational =  -7.334472468201543
epoch =  192 batch =  81 / 101 loss_uniform =  -7.389992078145345  loss_variational =  -7.34266123359586
epoch =  192 batch =  91 / 101 loss_uniform =  -7.402824129377093  loss_variational =  -7.358514696687132
epoch =  192 batch =  101 / 101 loss_uniform =  -7.577186046260418  loss_variational =  -7.515251891447766
epoch =  193 batch =  1 / 101 loss_uniform =  -6.987879276275635  loss_variational =  -6.824455261230469
epoch =  193 batch =  11 / 101 loss_uniform =  -7.188717495311391  loss_variational =  -7.147468913685191
epoch =  193 batch =  21 / 101 loss_uniform 

epoch =  199 batch =  51 / 101 loss_uniform =  -7.481681393642051  loss_variational =  -7.494746946821026
epoch =  199 batch =  61 / 101 loss_uniform =  -7.487934323607898  loss_variational =  -7.49907946977459
epoch =  199 batch =  71 / 101 loss_uniform =  -7.481984427277471  loss_variational =  -7.491020316808996
epoch =  199 batch =  81 / 101 loss_uniform =  -7.479757944742839  loss_variational =  -7.486784540576699
epoch =  199 batch =  91 / 101 loss_uniform =  -7.4674507811829285  loss_variational =  -7.473547689207307
epoch =  199 batch =  101 / 101 loss_uniform =  -7.531852708004489  loss_variational =  -7.5393625344380295


In [12]:
n_sample = 100000
with torch.no_grad():
    x_uniform = flow_uniform.sample(n_sample).cpu()
    x_variational = flow_variational.sample(n_sample).cpu()
x_data_plot = x_data_raw.cpu()[:n_sample,:]
x_weights_plot = x_weights_raw.cpu()[:n_sample]

In [17]:
x_variational[:,3]

tensor([7., 7., 5.,  ..., 7., 7., 6.])

In [20]:
x_data_plot[:,3]

tensor([ 5.,  0., 12.,  ..., 16., 14.,  9.])

In [21]:
plt.yscale("log")
bins = np.linspace(-0.5, 16.5, 18)
plt.hist(x_data_plot[:,3], histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins=bins, weights=x_weights_plot)
#plt.hist(x_uniform[:,3], edgecolor="red", histtype="step", bins = bins)
#plt.hist(x_variational[:,3], edgecolor="green", histtype="step", bins = bins)
plt.show()

KeyboardInterrupt: 

In [15]:
with torch.no_grad():
    x_uniform_dequantized, _ = flow_uniform._transform._transforms[0].forward(x_uniform)
    x_variational_dequantized, _ = flow_variational._transform._transforms[0].forward(x_variational)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
plt.yscale("log")
plt.hist(x_uniform_dequantized[:,2], histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins = 100)
plt.hist(x_variational_dequantized[:,2], edgecolor="red", histtype="step", bins = 100)
plt.show()